# Проект:

# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.  
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Задание 1. Исследование

<font style="background-color:#AFEEEE">**Шаг 1.** Открываем файл с данными /datasets/data.csv и печатаем первые строки для ознакомления.</font>

In [0]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(5)

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Сразу видим в трудовом стаже - отрицательные значения. Что это значит? Скорее всего ошибка выгрузки, надо разбираться.

Пока изучим общую информацию.

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [0]:
data.columns

Index(['children', 'days_employed', 'dob_days', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

В нашей таблице 21525 строк и есть пропущенные значения. Названия столбцов выглядят корректно.  
Сохраним количество строк в переменной и посчитаем количество пропущенных значений.

In [0]:
data_len = 21525
data.isna().sum()

children               0
days_employed       2174
dob_days               0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пробелы в столбцах с трудовым стажем ('days_employed') и доходом в месяц ('total_income').  
Можно приступать к Шагу 2.

<font style="background-color:#AFEEEE">**Шаг.2** Предобработка данных.</font>

Отфильтруем таблицу по пропущенным значениям и посмотрим на количество строк. 

In [0]:
nan_data = data[(data['days_employed'].isnull()) | (data['total_income'].isnull())]
nan_data.shape

(2174, 12)

Отфильтрованная таблица состоит из 2174 строки. Выходит, если в строке не указан трудовой стаж, в нем не указан и ежемесячный доход. Странно. Стаж и доход - необходимая информация для подачи заявки на кредит. Видимо данные не указаны не намеренно, а потеряны. Отсутствуют значения в 10% записей. 

Посмотрим на них внимательней.

In [0]:
nan_data.head(10)

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Госслужащие, сотрудники, пенсионеры.. Выгрузим весь список.

In [0]:
nan_data['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Список по типу занятости разгадки не дает, что ж.. Заменим отстутствующие значения на медианы по столбцами. Удалять строки с пропущенными значениями не будем.  
Перед нами стоит задача выяснить, как влияет семейное положение и количество детей на возврат кредита в срок. Отсутствующие стаж и доход нам никак не помешают.

In [0]:
days_employed_mean = data['days_employed'].mean()
total_income_mean = data['total_income'].mean()
data['days_employed'] = data['days_employed'].fillna(value=days_employed_mean)
data['total_income'] = data['total_income'].fillna(value=total_income_mean)
data.isna().sum()

children            0
days_employed       0
dob_days            0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<font style="color:green; font-size:200%">✓</font> Ок. Теперь набор данных полный. 

Но мы уже заметили в самом начале, что в стаже есть отрицательные значения.

Приведем сразу рабочий стаж к цельночисленном положительным значениям и заодно посмотрим на него в годах.

In [0]:
data['days_employed'] = data['days_employed'].astype('int').abs()
data['years_employed'] = (data['days_employed'] / 365).astype('int')
data[['children', 'years_employed', 'dob_days', 'education', 'family_status', 'gender', 'income_type', 'debt', 'total_income', 'purpose']].head()

,children,years_employed,dob_days,education,family_status,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,женат / замужем,F,сотрудник,0,253875.639453,покупка жилья
1,1,11,36,среднее,женат / замужем,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15,33,Среднее,женат / замужем,M,сотрудник,0,145885.952297,покупка жилья
3,3,11,32,среднее,женат / замужем,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932,53,среднее,гражданский брак,F,пенсионер,0,158616.077870,сыграть свадьбу


В среднем при выходе на пенсию трудовой стаж человека составляет в нашей стране - 35 лет. Посмотрим, сколько в таблице значений больше этой цифры.

In [0]:
"{:.0%}".format(data.loc[data.loc[:,'years_employed'] > 35]['years_employed'].count() / data_len)

'26%'

Цифра не маленькая. Целых 26% от всех подавших заявку на кредит отработали дольше среднего трудового стажа, собственного возраста, а некоторые даже по несколько жизней.

Проверим наши данные на дубликаты и удалим их, если они есть.

In [0]:
data.duplicated().sum()

54

In [0]:
data = data.drop_duplicates()
data.duplicated().sum()

0

<font style="color:green; font-size:200%">✓</font> От дубликатов избавились.

 <font style="background-color:#ffff00"> >>> Проверим количественные переменные на наличие ошибок и "причешем" их. <<< </font>
    
 Начнем с детей:

In [0]:
data['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Есть отрицательное количество детей и даже 20 детей. И это не разовый случай. Отрицательных детей сделаем положительными.  
А вот 20 детей это за пределами возможностей обычной человеческой особи, скорее всего ошибка в данных, эти строки нужно удалить.

In [0]:
data['children'] = data['children'].replace(-1, 1)
data = data[ data['children'] != 20]
data['children'].value_counts()

0    14107
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

С количеством детей разобрались. Они для нашего исследования важны.  

Посмотрим на возраст:

In [0]:
dob_days_data = data['dob_days'].value_counts().reset_index().sort_values(by='index')
dob_days_data.head()

,index,dob_days
47,0,100
54,19,14
52,20,51
46,21,110
43,22,183


In [0]:
dob_days_data.tail()

,index,dob_days
51,71,58
53,72,33
55,73,8
56,74,6
57,75,1


Кто-то возраст не указал вовсе, кто-то рассчитывает получить кредит в 75 лет. Ничего криминального в этом столбце данных не обнаружено.

Проверим на всякий случай, не встречается ли у нас в данных отрицательный доход. И приведем данные с плавающей точкой к целым числам.

In [0]:
data.loc[data.loc[:,'total_income'] < 0]['total_income'].count()

0

In [0]:
data['total_income'] = data['total_income'].astype('int')
data.head(1)

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23


<font style="color:green; font-size:200%">✓</font> Заменили вещественные типы данных на цельночисленные.

 <font style="background-color:#ffff00"> >>> Для порядка пробежимся по столбцам с категориальными данными. <<< </font>

Образование:

In [0]:
data['education'].value_counts()

среднее                13653
высшее                  4698
СРЕДНЕЕ                  770
Среднее                  705
неоконченное высшее      666
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Причешем.

In [0]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15128
высшее                  5237
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

Семейное положение:

In [0]:
data['family_status'].value_counts()

женат / замужем          12295
гражданский брак          4151
Не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

Тут все четко.  
Пол?

In [0]:
data['gender'].value_counts()

F      14142
M       7252
XNA        1
Name: gender, dtype: int64

Есть один неопределившийся! Удалим эту строку.

In [0]:
data = data[data['gender'] != 'XNA']

Тип занятости:

In [0]:
data['income_type'].value_counts()

сотрудник          11048
компаньон           5057
пенсионер           3828
госслужащий         1455
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [0]:
data['debt'].value_counts()

0    19661
1     1733
Name: debt, dtype: int64

И наконец Цель получения кредита:

In [0]:
data['purpose'].value_counts()

свадьба                                   792
на проведение свадьбы                     769
сыграть свадьбу                           765
операции с недвижимостью                  674
покупка коммерческой недвижимости         659
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          624
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

Варинатов целей много, но по сути их можно светсти к нескольким категориям.

Соберем уникальные цели в отдельную таблицу.

In [0]:
purpose_data = data['purpose'].value_counts().reset_index().sort_values(by='index')
purpose_data.head()

,index,purpose
24,автомобили,477
22,автомобиль,493
31,высшее образование,447
29,дополнительное образование,457
9,жилье,642


Лемматизируем цели, чтобы в итоге с помощью словаря категоризировать исходную таблицу.

In [0]:
from pymystem3 import Mystem
m = Mystem()
def lemmatize_purpose(data):
    index = data['index']
    lemma = m.lemmatize(index)
    return lemma

purpose_data['index_modified'] = purpose_data.apply(lemmatize_purpose, axis=1)  

purpose_data.head()

,index,purpose,index_modified
24,автомобили,477,"[автомобиль, \n]"
22,автомобиль,493,"[автомобиль, \n]"
31,высшее образование,447,"[высокий, , образование, \n]"
29,дополнительное образование,457,"[дополнительный, , образование, \n]"
9,жилье,642,"[жилье, \n]"


Посмотрим, какие леммы мы сможем исользовать как категории.

In [0]:
list_purpose_data = purpose_data['index_modified'].sum()

from collections import Counter
print(Counter(list_purpose_data))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'на': 4, 'свой': 4, 'операция': 4, 'высокий': 3, 'свадьба': 3, 'получение': 3, 'строительство': 3, 'дополнительный': 2, 'заниматься': 2, 'коммерческий': 2, 'жилой': 2, 'для': 2, 'сделка': 2, 'подержать': 1, 'проведение': 1, 'со': 1, 'сдача': 1, 'семья': 1, 'приобретение': 1, 'профильный': 1, 'ремонт': 1, 'подержанный': 1, 'собственный': 1, 'сыграть': 1})


Создаем список категорий:

In [0]:
purpose_id = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'ремонт', 'жилье',]

Скорректируем функцию для полной таблицы и добавим столбец с категориями.

In [0]:
def categorize_purpose(data):
    purpose = data['purpose']
    lemma = m.lemmatize(purpose)
    if purpose_id[0] in lemma:
        return purpose_id[0]
    if purpose_id[1] in lemma:
        return purpose_id[1]
    if purpose_id[2] in lemma:
        return purpose_id[2]
    if purpose_id[3] in lemma:
        return purpose_id[3]
    if purpose_id[4] in lemma:
        return purpose_id[4]
    if purpose_id[5] in lemma:
        return purpose_id[0]
    else: 
        return "категория не назначена"

data['category'] = data.apply(categorize_purpose, axis=1)

data[['purpose', 'category']].head()

,purpose,category
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба


Для того чтобы было проще ориентироваться в клиентских запросах, категоризируем клиентов банка по возрасту, наличию детей и группе дохода.

In [0]:
def age_group(dob_days):
        if dob_days <= 25:
                return '19-25'
        if dob_days <= 35:
                return '26-35'
        if dob_days <= 45:
                return '36-45'
        if dob_days <= 55:
                return '46-55'
        if dob_days > 55:
                return '56+'
            
data['age_group'] = data['dob_days'].apply(age_group)

data[['dob_days', 'age_group']].head()

,dob_days,age_group
0,42,36-45
1,36,36-45
2,33,26-35
3,32,26-35
4,53,46-55


In [0]:
def children_group(children):
        if children >= 1:
                return 'есть дети'
        else:
                return 'детей нет'
            
data['children_group'] = data['children'].apply(children_group)

data[['children', 'children_group']].head()

,children,children_group
0,1,есть дети
1,1,есть дети
2,0,детей нет
3,3,есть дети
4,0,детей нет


Создадим группы дохода:
    
    до 50 000
    50 000 - 80 000
    80 000 - 100 000
    100 000 - 250 000
    250 000 - 500 000
    500 000 - 1 000 000
    1 000 000 и более

In [0]:
def income_group(total_income):
        if total_income <= 50000:
                return 'до 50 000'
        if total_income <= 80000:
                return '50 000 - 80 000'
        if total_income <= 100000:
                return '80 000 - 100 000'
        if total_income <= 250000:
                return '100 000 - 250 000'
        if total_income <= 500000:
                return '250 000 - 500 000'
        if total_income <= 1000000:
                return '500 000 - 1 000 000'
        if total_income > 1000000:
                return '1 000 000 и более'
            
data['income_group'] = data['total_income'].apply(income_group)

data[['total_income', 'income_group']].head()

,total_income,income_group
0,253875,250 000 - 500 000
1,112080,100 000 - 250 000
2,145885,100 000 - 250 000
3,267628,250 000 - 500 000
4,158616,100 000 - 250 000


## Задание 2. Выводы

Посмотрим на таблицу еще раз:

In [0]:
data[['children_group', 'years_employed', 'age_group', 'education', 'family_status', 'gender', 'income_type', 'debt', 'income_group', 'category']].head()

,children_group,years_employed,age_group,education,family_status,gender,income_type,debt,income_group,category
0,есть дети,23,36-45,высшее,женат / замужем,F,сотрудник,0,250 000 - 500 000,недвижимость
1,есть дети,11,36-45,среднее,женат / замужем,F,сотрудник,0,100 000 - 250 000,автомобиль
2,детей нет,15,26-35,среднее,женат / замужем,M,сотрудник,0,100 000 - 250 000,недвижимость
3,есть дети,11,26-35,среднее,женат / замужем,M,сотрудник,0,250 000 - 500 000,образование
4,детей нет,932,46-55,среднее,гражданский брак,F,пенсионер,0,100 000 - 250 000,свадьба


* <font style="background-color:#DCDCDC">Есть ли зависимость между наличием детей и возвратом кредита в срок?</font>

Напечатаем количество задолженностей по кредиту в зависимости наличия детей.

In [0]:
children_data_pivot = data.pivot_table(index=['children_group'], columns=['gender'], values='debt', aggfunc={sum, len})
children_data_pivot['sum'] = (children_data_pivot['sum'] / children_data_pivot['len'] *100).astype('int')
children_data_pivot['sum']

gender,F,M
children_group,,
детей нет,6,10
есть дети,8,10


Получаемые проценты показывают, что мужчины из нашего датасета всегда отдают кредиты хуже, чем женщины - вне зависимости от количества детей.

* <font style="background-color:#FFDAB9">Есть ли зависимость между наличием детей и возвратом кредита в срок?</font>

In [0]:
result = data.pivot_table(index=['children_group'], columns=['debt'], values = ['dob_days'], aggfunc = 'count')
result.columns = ['Не-должники', 'Должники']
result['Всего'] = result['Не-должники'] + result['Должники']
result['% должников'] = (result['Должники'] / result['Всего']*100).round(2)
result

,Не-должники,Должники,Всего,% должников
children_group,,,,
детей нет,13043,1063,14106,7.54
есть дети,6618,670,7288,9.19


**Клиенты с детьми реже берут кредиты и хуже их возвращают**. 

* <font style="background-color:#FFDAB9">Есть ли зависимость между семейным положением и возвратом кредита в срок?</font>

In [0]:
result = data.pivot_table(index=['family_status'], columns=['debt'], values = ['dob_days'], aggfunc = 'count')
result.columns = ['Не-должники', 'Должники']
result['Всего'] = result['Не-должники'] + result['Должники']
result = result[result['Должники'] >= 100]
result['% должников'] = (result['Должники'] / result['Всего']*100).round(2)
result

,Не-должники,Должники,Всего,% должников
family_status,,,,
Не женат / не замужем,2528,273,2801,9.75
гражданский брак,3765,385,4150,9.28
женат / замужем,11367,928,12295,7.55


Чаще других групп берут кредиты **люди состоящие в браке**, в этой группе **самый низкий процент должников**.  
**Самый выскоий** процент должников **среди холостяков**.

<font color="grey">*К анализу принимаются группы от 100 чел.*</font>

* <font style="background-color:#FFDAB9">Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</font>

In [0]:
result = data.pivot_table(index=['income_group'], columns=['debt'], values = ['dob_days'], aggfunc = 'count')
result.columns = ['Не-должники', 'Должники']
result['Всего'] = result['Не-должники'] + result['Должники']
result = result[result['Должники'] >= 100]
result['% должников'] = (result['Должники'] / result['Всего']*100).round(2)
result

,Не-должники,Должники,Всего,% должников
income_group,,,,
100 000 - 250 000,12956,1185,14141,8.38
250 000 - 500 000,2399,180,2579,6.98
50 000 - 80 000,1747,151,1898,7.96
80 000 - 100 000,2002,180,2182,8.25


**Самый высокий** процент должников в группе **"100 000 - 250 000"**, а **самый низкий** - в группе **"250 000 - 500 000"**.

<font color="grey">*К анализу принимаются группы от 100 чел.*</font>

* <font style="background-color:#FFDAB9">Как разные цели кредита влияют на его возврат в срок?</font>

In [0]:
result = data.pivot_table(index=['category'], columns=['debt'], values = ['dob_days'], aggfunc = 'count')
result.columns = ['Не-должники', 'Должники']
result['Всего'] = result['Не-должники'] + result['Должники']
result = result[result['Должники'] >= 100]
result['% должников'] = (result['Должники'] / result['Всего']*100).round(2)
result

,Не-должники,Должники,Всего,% должников
category,,,,
автомобиль,3891,401,4292,9.34
недвижимость,9427,745,10172,7.32
образование,3630,369,3999,9.23
свадьба,2143,183,2326,7.87


**Самый высокий** процент должников в категории **"автомобиль"**, а **самый низкий** - в категории **"недвижимость"**.

<font color="grey">*К анализу принимаются группы от 100 чел.*</font>

<font style="background-color:#AFEEEE">**Шаг 2.** Общий вывод.</font>

Рекомендуем привлекать больше клиентов из групп, с минимальным процентом должников.

Из полученных выше данных можно сделать вывод, что с большей вероятностью кредит банку отдаст вовремя заемщик состоящий в браке, без детей, с доходом 250 000 - 500 000, который хочет приобрести недвижимость.